## Extracción, Tratamiento y Carga de datos (ETL)

### Objetivo

El objetivo de este notebook es realizar un proceso de ETL de los datasets . Como primer paso los datos se prepararán y limpiarán para su posterior análisis.

### Archivo de Datos: `homicidios.xlsx` y `lesiones.xlsx`

El archivo contiene información sobre los siniestros viales: homicidio (victimas fatales) y lesiones (victimas no fatales).

El detalle de los datos se encuentran en el siguiente [LINK](https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/NOTAS_HOMICIDIOS_SINIESTRO_VIAL.pdf) con información de las variables.


### 1. Carga de datos

LIBRERÍAS

In [3]:
import pandas as pd
import numpy as np
#pip install openpyxl


In [4]:
# Cargamos los archivos
df_homicidios = pd.read_excel("homicidios.xlsx", sheet_name= ['HECHOS','VICTIMAS'])
df_lesiones = pd.read_excel("lesiones.xlsx", sheet_name= ['HECHOS','VICTIMAS'])


### 2. Exploramos el dataframe

#### 2.1 Homicidios - Hechos

Se explora la data del archivo `homicidios.xlsx` específicamente la hoja HECHOS.
Contine una fila ppr hecho con id único y las variables asociadas.

In [5]:
df_homicidios_hechos = df_homicidios['HECHOS']
print(df_homicidios_hechos.shape)
df_homicidios_hechos.head(2)

(696, 21)


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


Revisamos la estructura y la calidad de los datos del Dataframe "df_homicidios_hechos"

In [6]:
#Con .info() obtenemos una visión general rápida del DataFrame
df_homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

De las 21 columnas, 4 tienen valores nulos ('Calle', 'Altura', 'Cruce' y 'Dirección Normalizada'). Nota: Algunos valores en las columnas son SD: Sin Dato. Se considera como NaN.

In [7]:
# Backup
df1 = df_homicidios_hechos.copy()

LIMPIEZA Y PREPROCESAMIENTO DE LOS DATOS

Eliminamos columnas con la misma información: Quitamos algunas columnas que nos brinda la misma información, se busca mejorar el renidmiento y reducir redundancia:

|Columnas a eliminar|Justificación|
|----|----|
|'AAAA', 'MM', 'DD'| Esta información se encuentra también en la variable 'FECHA'|
|'HORA'|Esta información se encuentra en HH. En este caso nos centraremos en la franja horaria entera del siniestro para simplificar cálculos.|
|'PARTICIPANTES'| Esta información se encuentra en las columnas VICTIMA y ACUSADO|
|'XY (CABA)'|Se opta por trabajar con las coordenas xy|
|'Dirección Normalizada'|Se opta por trabajar con las coordenadas|


In [8]:
columnas_a_eliminar =["AAAA","MM","DD","HORA","PARTICIPANTES","XY (CABA)",'Dirección Normalizada']
df1.drop(columnas_a_eliminar, axis=1, inplace=True)


Estandarizamos nulos

In [9]:
# Reemplaza 'SD' por NaN en todo el DataFrame
df1.replace('SD', np.nan, inplace=True)


In [10]:
# Contamos los valores nulos
columnas_con_nulos = df1.isnull().sum()

# Filtra solo las columnas que tienen al menos un valor nulo
columnas_con_nulos = columnas_con_nulos[columnas_con_nulos > 0]
print("Columnas con valores nulos:")
print(columnas_con_nulos)

Columnas con valores nulos:
HH                   1
LUGAR_DEL_HECHO      1
Calle                1
Altura             567
Cruce              171
VICTIMA              9
ACUSADO             23
dtype: int64


Renombramos columnas

In [11]:
# Para mejorar la consistencia y la legibilidad de los nombres de las columnas en un conjunto de datos.
nuevos_nombres = {'ID': 'ID_siniestro', 'N_VICTIMAS': 'Nro_victimas', 'FECHA':'Fecha','HH':'Franja_horaria','TIPO_DE_CALLE':'Tipo_calle','COMUNA':'Comuna','pos x': 'Longitud','pos y':'Latitud','VICTIMA':'Vehiculo_Victima','ACUSADO':'Vehiculo_Acusado' }
df1.rename(columns=nuevos_nombres, inplace=True)

Completamos datos de Longitud y Latitud

In [12]:
# Filtra solo columnas sin coordenadas
df_w_coord = df1[(df1['Longitud'] == '.') & (df1['Latitud'] == '.')]
df_w_coord.shape

(12, 14)

Completamos datos de latitud y longitud con datos de la web ([Google Maps](https://support.google.com/maps/answer/18539?hl=es-MX&co=GENIE.Platform%3DDesktop#:~:text=C%C3%B3mo%20obtener%20las%20coordenadas%20de,decimal%20en%20la%20parte%20superior) y de [Comunas Buenos Aires](https://www.argentina.gob.ar/caba/comunas))

In [13]:
df1.loc[df1['ID_siniestro'] == '2020-0039', ['Longitud', 'Latitud']] = [-58.40294110948785, -34.66235319327016]
df1.loc[df1['ID_siniestro'] == '2021-0023', ['Longitud', 'Latitud']]= [-58.356652005326985, -34.628377264063815]
df1.loc[df1['ID_siniestro'] == '2020-0026', ['Longitud', 'Latitud']]= [-58.42560076032446, -34.5558736017778] 
df1.loc[df1['ID_siniestro'] == '2018-0039', ['Longitud', 'Latitud']]= [-58.41514789314846, -34.56342648312684] 
df1.loc[df1['ID_siniestro'] == '2017-0140', ['Longitud', 'Latitud']]= [-58.380775770, -34.622228850] 
df1.loc[df1['ID_siniestro'] == '2017-0051', ['Longitud', 'Latitud']]= [-58.46627594679307, -34.650250816993506] 
df1.loc[df1['ID_siniestro'] == '2017-0050', ['Longitud', 'Latitud']]= [-58.46570592606029, -34.649354633269255] 
df1.loc[df1['ID_siniestro'] == '2017-0042', ['Longitud', 'Latitud']]= [-58.42364891764647, -34.55707759622746] 
df1.loc[df1['ID_siniestro'] == '2016-0174', ['Longitud', 'Latitud']]= [-58.454422502287365, -34.63738948123826] # Aproximado
df1.loc[df1['ID_siniestro'] == '2016-0151', ['Longitud', 'Latitud']]= [-58.50801713, -34.65177305] # Aproximado
df1.loc[df1['ID_siniestro'] == '2016-0136', ['Longitud', 'Latitud']]= [-58.356652005326985, -34.628377264063815]
df1.loc[df1['ID_siniestro'] == '2016-0052', ['Longitud', 'Latitud']]= [-58.46435413701649, -34.534604784866964] 



Los valores de las *comunas* se pueden aproximar con las coordenadas u otra información de la zona


In [14]:
# Paras las comunas = 0
df1.loc[df1['ID_siniestro'] == '2016-0151', ['Comuna']] = [9] # Segun Calle similares
df1.loc[df1['ID_siniestro'] == '2016-0174', ['Comuna']] = [7] # Segun el lugar del hecho


Completamos la Calle faltante

In [15]:
df1.loc[df1['ID_siniestro'] == '2016-0151', 'Calle'] = 'Murguiondo'

Establecemos el formato de las columnas

In [16]:
# Damos tipo de datos adecuado
# Justificacion: Reducir el uso de memoria y mejorar el rendimiento.
df1['Franja_horaria'] = pd.to_numeric(df1['Franja_horaria'], errors='coerce').astype('Int64')
df1['Franja_horaria'] = df1['Franja_horaria'].astype('category')
df1['Tipo_calle'] = df1['Tipo_calle'].astype('category')
df1['Comuna'] = df1['Comuna'].astype('category')
df1['Vehiculo_Victima'] = df1['Vehiculo_Victima'].astype('category')
df1['Vehiculo_Acusado'] = df1['Vehiculo_Acusado'].astype('category')
# A este punto Franja_horaria, Comuna y Nro_victimas son int

Filtramos columnas necesarias

In [17]:
columnas_a_eliminar2 =['LUGAR_DEL_HECHO','Cruce','Altura','Calle']
df1.drop(columnas_a_eliminar2, axis=1, inplace=True)

Eliminamos duplicados

In [18]:
df1.duplicated().sum()

0

Validación de valores: 

Segun el diccionario de hechos hay validaciones de los valores que puede tomar el 'Tipo_calle', 'Vehiculo_Victima', 'Vehiculo_Acusado', '

In [19]:
df1['Tipo_calle'].value_counts()

Tipo_calle
AVENIDA      429
CALLE        136
AUTOPISTA     66
GRAL PAZ      65
Name: count, dtype: int64

In [20]:
df1['Vehiculo_Victima'].value_counts()

Vehiculo_Victima
MOTO           295
PEATON         264
AUTO            83
BICICLETA       29
CARGAS           7
PASAJEROS        5
MOVIL            2
OBJETO FIJO      1
PEATON_MOTO      1
Name: count, dtype: int64

In [21]:
# Reemplazamos por 'Otro' los distintos valores de cada variable
df1['Vehiculo_Victima'].replace(['PEATON_MOTO', 'OBJETO FIJO'], 'Otro', inplace=True)


In [22]:
df1['Vehiculo_Acusado'].value_counts()

Vehiculo_Acusado
AUTO           204
PASAJEROS      173
CARGAS         146
OBJETO FIJO     62
MOTO            57
MULTIPLE        17
BICICLETA        7
OTRO             6
TREN             1
Name: count, dtype: int64

Resumen

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_siniestro      696 non-null    object        
 1   Nro_victimas      696 non-null    int64         
 2   Fecha             696 non-null    datetime64[ns]
 3   Franja_horaria    695 non-null    category      
 4   Tipo_calle        696 non-null    category      
 5   Comuna            696 non-null    category      
 6   Longitud          696 non-null    object        
 7   Latitud           696 non-null    object        
 8   Vehiculo_Victima  687 non-null    category      
 9   Vehiculo_Acusado  673 non-null    category      
dtypes: category(5), datetime64[ns](1), int64(1), object(3)
memory usage: 33.0+ KB


In [24]:
df1.head(2)

,ID_siniestro,Nro_victimas,Fecha,Franja_horaria,Tipo_calle,Comuna,Longitud,Latitud,Vehiculo_Victima,Vehiculo_Acusado
0,2016-0001,1,2016-01-01,4,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,1,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS


#### 2.2 Homicidios - Victimas

Se explora la data del archivo `homicidios.xlsx` específicamente la hoja VICTIMAS. Contiene una fila por cada víctima de los hechos 

In [25]:
df_homicidios_victimas = df_homicidios['VICTIMAS']
print(df_homicidios_victimas.shape)
df_homicidios_victimas.head(2)

(717, 10)


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


In [26]:
df_homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


No hay nulos. Nota: Algunos valores en las columnas son SD (Sin Dato). Se considera como NaN.

In [27]:
# Backup
df2 = df_homicidios_victimas.copy()

LIMPIEZA Y PREPROCESAMIENTO DE LOS DATOS

Estandarizamos nulos

In [28]:
# Reemplaza 'SD' por NaN en todo el DataFrame
df2.replace('SD', np.nan, inplace=True)


In [29]:
# Contamos los valores nulos
columnas_con_nulos = df2.isnull().sum()

# Filtra solo las columnas que tienen al menos un valor nulo
columnas_con_nulos = columnas_con_nulos[columnas_con_nulos > 0]
print("Columnas con valores nulos:")
print(columnas_con_nulos)

Columnas con valores nulos:
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64


Eliminamos columnas con la misma información: Quitamos algunas columnas que nos brinda la misma información, se busca mejorar el rendimiento y reducir redundancia:

|Columnas a eliminar|Justificación|
|----|----|
|'AAAA', 'MM', 'DD'| Esta información se encuentra también en la variable 'FECHA'|
|'FECHA' |Variable se puede encontrar en HECHOS a traves del ID del siniestro |
|"VICTIMA"|Se tiene la misma informacion en HECHOS|
|"FECHA_FALLECIMIENTO"|Nos enfocamos en la fatalidad|

In [30]:
columnas_a_eliminar =["AAAA","MM","DD","FECHA","VICTIMA","FECHA_FALLECIMIENTO"]
df2.drop(columnas_a_eliminar, axis=1, inplace=True)


Renombramos columnas

In [31]:
# Para mejorar la consistencia y la legibilidad de los nombres de las columnas en un conjunto de datos.
nuevos_nombres = {'ID_hecho': 'ID_siniestro', 'ROL': 'Rol_vehiculo', 'SEXO':'Sexo', 'EDAD':'Edad'}
df2.rename(columns=nuevos_nombres, inplace=True)

In [32]:
df2.head(2)

,ID_siniestro,Rol_vehiculo,Sexo,Edad
0,2016-0001,CONDUCTOR,MASCULINO,19.0
1,2016-0002,CONDUCTOR,MASCULINO,70.0


Establecemos el formato de las columnas

In [33]:
# Damos tipo de datos adecuado
# Justificacion: Reducir el uso de memoria y mejorar el rendimiento.
df2['Edad'] = pd.to_numeric(df2['Edad'], errors='coerce').astype('Int64')
df2['Rol_vehiculo'] = df2['Rol_vehiculo'].astype('category')
df2['Sexo'] = df2['Sexo'].astype('category')
#df2['Vehiculo_Victima'] = df2['Vehiculo_Victima'].astype('category')

Etiqueta de que los datos son de victimas fatales

In [34]:
df2['gravedad'] = 'FATAL' #Terminología que se usa en el diccionario hechos del archivo lesiones

In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID_siniestro  717 non-null    object  
 1   Rol_vehiculo  706 non-null    category
 2   Sexo          711 non-null    category
 3   Edad          664 non-null    Int64   
 4   gravedad      717 non-null    object  
dtypes: Int64(1), category(2), object(2)
memory usage: 19.4+ KB


In [36]:
df2.tail()

,ID_siniestro,Rol_vehiculo,Sexo,Edad,gravedad
712,2021-0092,PEATON,FEMENINO,50,FATAL
713,2021-0093,PASAJERO_ACOMPAÑANTE,FEMENINO,18,FATAL
714,2021-0094,PASAJERO_ACOMPAÑANTE,FEMENINO,43,FATAL
715,2021-0095,CONDUCTOR,MASCULINO,27,FATAL
716,2021-0096,CONDUCTOR,MASCULINO,60,FATAL


#### 2.3 Lesiones - Hechos
Se explora la data del archivo `lesiones.xlsx` específicamente la hoja HECHOS.

In [37]:
df_lesiones_hechos = df_lesiones['HECHOS']
print(df_lesiones_hechos.shape)
df_lesiones_hechos.head(2)

(23785, 27)


,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD


In [38]:
df_lesiones_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

De las 27 columnas, 10 tienen valores nulos ('franja_hora', 'direccion_normalizada', 'comuna', 'calle', 'altura', 'cruce', 'ciclista', 'geocodificacion_CABA', 'longitud', 'latutid'). Nota: Algunos valores en las columnas son SD: Sin Dato. Se considera como NaN.

In [39]:
# Backup
df3 = df_lesiones_hechos.copy()

Estandarizamos nulos

In [40]:
# Reemplaza 'SD' por NaN en todo el DataFrame
df3.replace(['SD','sd'], np.nan, inplace=True)


In [41]:
# Contamos los valores nulos
columnas_con_nulos = df3.isnull().sum()

# Filtra solo las columnas que tienen al menos un valor nulo
columnas_con_nulos = columnas_con_nulos[columnas_con_nulos > 0]
print("Columnas con valores nulos:")
print(columnas_con_nulos)

Columnas con valores nulos:
hora                         9
franja_hora                  9
direccion_normalizada    10868
comuna                    1015
tipo_calle               11045
otra_direccion           18295
calle                    10918
altura                   11014
cruce                    14378
geocodificacion_CABA      1252
longitud                  1471
latutid                   1471
victima                  10733
acusado                  15288
moto                      8604
auto                     12636
transporte_publico       11894
camion                   12801
ciclista                 11446
gravedad                 23139
dtype: int64


LIMPIEZA Y PREPROCESAMIENTO DE LOS DATOS

Eliminamos columnas con la misma información: Quitamos algunas columnas que nos brinda la misma información, se busca mejorar el renidmiento y reducir redundancia:

|Columnas a eliminar|Justificación|
|----|----|
|'aaaa', 'mm', 'dd'| Esta información se encuentra también en la variable 'fecha'|
|'hora'|Esta información se encuentra en franja_hora. En este caso nos centraremos en la franja horaria entera|
|'direccion_normalizada','cruce','calle'|Se usa coordenadas para definir la ubicacion en su lugar|
|'otra_direccion','altura','geocodificacion_CABA'| Se usa coordenadas para definir la ubicacion en su lugar |

In [42]:
columnas_a_eliminar =["aaaa","mm","dd","hora","altura","direccion_normalizada","otra_direccion","calle","cruce","geocodificacion_CABA","participantes"]
df3.drop(columnas_a_eliminar, axis=1, inplace=True)


Reemplazamos los datos con 1 para indicar participación del vehículo

In [43]:
# Supongamos que df es tu DataFrame
columnas_a_reemplazar = ['moto', 'auto', 'camion', 'ciclista', 'transporte_publico']

# Reemplaza 'x' por 1 en las columnas seleccionadas
df3[columnas_a_reemplazar] = df3[columnas_a_reemplazar].replace('x', 1)

Renombramos columnas

In [44]:
# Para mejorar la consistencia y la legibilidad de los nombres de las columnas en un conjunto de datos.
nuevos_nombres = {'id': 'ID_siniestro', 'n_victimas': 'Nro_victimas', 'fecha':'Fecha','franja_hora':'Franja_horaria','comuna':'Comuna','tipo_calle':'Tipo_calle','longitud': 'Longitud','latutid':'Latitud','victima':'Vehiculo_Victima','acusado':'Vehiculo_Acusado' }
df3.rename(columns=nuevos_nombres, inplace=True)

In [45]:
#Estandarizamos la representación de los nulos en el dataframe
df3 = df3.fillna(value=np.nan) #df3.isnull().sum()

Normalizamos valores

* Variable: gravedad

In [46]:
df3['gravedad'] = df3['gravedad'].replace('grave', 'GRAVE')

In [47]:
df3['gravedad'].fillna('LEVE', inplace=True)

In [48]:
df3['gravedad'].unique() # 3 tipos segun el diccionario de hechos

array(['LEVE', 'GRAVE', 'FATAL'], dtype=object)

* Variable: Comuna

In [49]:
df3['Comuna'].unique()

array([14, 8, 7, 3, 15, 11, 9, 1, 4, 12, 2, 5, 6, 13, 10,
       'No Especificada', nan], dtype=object)

In [50]:
df3['Comuna'].replace('No Especificada', np.nan, inplace=True)

* Variable: Nro_victimas

In [51]:
df3['Nro_victimas'].unique()

array([ 1,  4,  2,  3,  5, 10,  6, 16,  8,  0,  7], dtype=int64)

In [52]:
df3[df3['Nro_victimas']==0]

,ID_siniestro,Nro_victimas,Fecha,Franja_horaria,Comuna,Tipo_calle,Longitud,Latitud,Vehiculo_Victima,Vehiculo_Acusado,moto,auto,transporte_publico,camion,ciclista,gravedad
9928,PNA-2019-0005246,0,2019-12-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LEVE


In [53]:
df3.loc[df3['ID_siniestro'] == 'PNA-2019-0005246', 'Nro_victimas'] = 1 # Le damos el valor de 1 por tener una victima


* Variable: Ciclista

In [54]:
# segun el diccionario de datos: 
# es CICLISTA si hay participación de algún ciclista, sea víctima u acusado

# Filtrar según las condiciones 
filtro = (df3['Vehiculo_Victima'] == 'CICLISTA') | (df3['Vehiculo_Acusado'] == 'CICLISTA') & pd.isna(df3['ciclista'])

# Reemplazar los valores nulos por 1
df3.loc[filtro, 'ciclista'] = df3.loc[filtro, 'ciclista'].fillna(1)

Establecemos formato de las columnas

In [55]:
df3['Franja_horaria'] = pd.to_numeric(df3['Franja_horaria'], errors='coerce').astype('Int64')
df3['Comuna'] = pd.to_numeric(df3['Comuna'], errors='coerce').astype('Int64')

df3['Franja_horaria'] = df3['Franja_horaria'].astype('category')
df3['Tipo_calle'] = df3['Tipo_calle'].astype('category')
df3['Comuna'] = df3['Comuna'].astype('category')
df3['Vehiculo_Victima'] = df3['Vehiculo_Victima'].astype('category')
df3['Vehiculo_Acusado'] = df3['Vehiculo_Acusado'].astype('category')

In [56]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ID_siniestro        23785 non-null  object  
 1   Nro_victimas        23785 non-null  int64   
 2   Fecha               23785 non-null  object  
 3   Franja_horaria      23776 non-null  category
 4   Comuna              22328 non-null  category
 5   Tipo_calle          12740 non-null  category
 6   Longitud            22314 non-null  object  
 7   Latitud             22314 non-null  object  
 8   Vehiculo_Victima    13052 non-null  category
 9   Vehiculo_Acusado    8497 non-null   category
 10  moto                15181 non-null  float64 
 11  auto                11149 non-null  float64 
 12  transporte_publico  11891 non-null  float64 
 13  camion              10984 non-null  float64 
 14  ciclista            12363 non-null  float64 
 15  gravedad            23785 non-null  

In [57]:
df3.head()

,ID_siniestro,Nro_victimas,Fecha,Franja_horaria,Comuna,Tipo_calle,Longitud,Latitud,Vehiculo_Victima,Vehiculo_Acusado,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,NaN,NaN,NaN,NaN,1.0,LEVE
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.44351,-34.669125,AUTO,NaN,NaN,1.0,NaN,NaN,NaN,LEVE
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,8,NaN,-58.468335,-34.677556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LEVE
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,7,NaN,-58.437425,-34.647349,PEATON,NaN,1.0,NaN,NaN,NaN,NaN,LEVE
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,3,NaN,-58.398225,-34.604579,AUTO,NaN,NaN,NaN,1.0,NaN,NaN,LEVE


#### 2.4 Lesiones - Victimas
Se explora la data del archivo `lesiones.xlsx` específicamente la hoja VICTIMAS.

In [58]:
df_lesiones_victimas = df_lesiones['VICTIMAS']
print(df_lesiones_victimas.shape)
df_lesiones_victimas.head(2)

(27605, 9)


,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD


Vemos el resumen del dataframe

In [59]:
df_lesiones_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


In [60]:
df4 = df_lesiones_victimas.copy()

In [61]:
# Reemplaza 'SD' por NaN en todo el DataFrame
df4.replace(['SD','sd'], np.nan, inplace=True)


Normalizamos nombres

In [62]:
# Reemplazar 'VARON' por 'Masculino' y 'Mujer' por 'Femenino'
df4['SEXO'] = df4['SEXO'].str.lower().str.strip().replace({'varon': 'MASCULINO', 'mujer': 'FEMENINO'})

In [63]:
df4['SEXO'].unique()

array(['MASCULINO', nan, 'FEMENINO'], dtype=object)

LIMPIEZA Y PREPROCESAMIENTO DE LOS DATOS

Eliminamos columnas con la misma información: Quitamos algunas columnas que nos brinda la misma información, se busca mejorar el renidmiento y reducir redundancia:

|Columnas a eliminar|Justificación|
|----|----|
|'aaaa', 'mm', 'dd'| Esta información se encuentra también en la variable 'fecha'|
|'GRAVEDAD','VEHICULO_VICTIMA'| Contine nulos y la información ya se encuentra en HECHOS|

In [64]:
columnas_a_eliminar =["AAA","MM","DD","VEHICULO_VICTIMA","GRAVEDAD","FECHA "]
df4.drop(columnas_a_eliminar, axis=1, inplace=True)


Renombramos columnas

In [65]:
# Para mejorar la consistencia y la legibilidad de los nombres de las columnas en un conjunto de datos.
nuevos_nombres = {'ID hecho': 'ID_siniestro', 'SEXO':'Sexo','EDAD_VICTIMA':'Edad' }
df4.rename(columns=nuevos_nombres, inplace=True)

In [66]:
df4.head()

,ID_siniestro,Sexo,Edad
0,LC-2019-0000053,MASCULINO,57
1,LC-2019-0000063,NaN,NaN
2,LC-2019-0000079,MASCULINO,NaN
3,LC-2019-0000082,MASCULINO,45
4,LC-2019-0000082,FEMENINO,45


Establecemos formato de las columnas

In [67]:
df4['Sexo'] = df4['Sexo'].astype('category')
df4['Edad'] = df4['Edad'].astype('category')

Resumen

In [68]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID_siniestro  27605 non-null  object  
 1   Sexo          25377 non-null  category
 2   Edad          23134 non-null  category
dtypes: category(2), object(1)
memory usage: 302.3+ KB


In [69]:
df4.head()

,ID_siniestro,Sexo,Edad
0,LC-2019-0000053,MASCULINO,57
1,LC-2019-0000063,NaN,NaN
2,LC-2019-0000079,MASCULINO,NaN
3,LC-2019-0000082,MASCULINO,45
4,LC-2019-0000082,FEMENINO,45


### 3. Cargar
En este paso se guarda los datos transformados en un formato csv y listos para su posterior análisis o uso.

Hasta el momento tenemos 4 dataframes trabajados: df1, df2, df3, df4

* homicidios >> HECHOS     (df1)
* homicidios >> VICTIMAS   (df2)
* lesiones >> HECHOS       (df3)
* lesiones >> VICTIMAS     (df4)

In [70]:
# Consolidamos la información 

# Fusiona los DataFrames en base a la columna 'ID_siniestro'
df_homicidios_final = pd.merge(df1, df2, on='ID_siniestro', how='left')
df_lesiones_final = pd.merge(df3, df4, on='ID_siniestro', how='left')

In [71]:
print(df_homicidios_final.shape)
df_homicidios_final.head(2)

(717, 14)


,ID_siniestro,Nro_victimas,Fecha,Franja_horaria,Tipo_calle,Comuna,Longitud,Latitud,Vehiculo_Victima,Vehiculo_Acusado,Rol_vehiculo,Sexo,Edad,gravedad
0,2016-0001,1,2016-01-01,4,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO,CONDUCTOR,MASCULINO,19,FATAL
1,2016-0002,1,2016-01-02,1,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS,CONDUCTOR,MASCULINO,70,FATAL


In [72]:
print(df_lesiones_final.shape)
df_lesiones_final.head(2)

(27602, 18)


,ID_siniestro,Nro_victimas,Fecha,Franja_horaria,Comuna,Tipo_calle,Longitud,Latitud,Vehiculo_Victima,Vehiculo_Acusado,moto,auto,transporte_publico,camion,ciclista,gravedad,Sexo,Edad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,NaN,NaN,NaN,NaN,1.0,LEVE,NaN,NaN
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.44351,-34.669125,AUTO,NaN,NaN,1.0,NaN,NaN,NaN,LEVE,MASCULINO,57


In [73]:
# Los archivos se almacenan en local 
df_homicidios_final.to_csv('homicidios_final.csv', index=False)
df_lesiones_final.to_csv('lesiones_final.csv', index=False)